In [1]:
import json
import requests

In [2]:

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
endpoint_url = "https://query.wikidata.org/sparql"

In [12]:
def get_results(query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_cocktails():
    # instance of IBA official cocktails
    cocktail_response = get_results("""
        SELECT ?cocktail ?cocktailLabel WHERE {
            ?cocktail wdt:P31 wd:Q2536409
            SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }
    """)
    return [
        { 'name': cocktail['cocktailLabel']['value'], 
            'uri:': cocktail['cocktail']['value'] } 
        for cocktail in cocktail_response['results']['bindings']
    ]

def get_ingredients(cocktail_url):
    ingredients_response = get_results("""
        
    """)



In [13]:
cocktails = get_cocktails()
print(cocktails[:5])

[{'name': 'Corpse Reviver #2', 'uri:': 'http://www.wikidata.org/entity/Q105872968'}, {'name': 'Illegal', 'uri:': 'http://www.wikidata.org/entity/Q105897287'}, {'name': 'Bloody Mary', 'uri:': 'http://www.wikidata.org/entity/Q207711'}, {'name': 'Zombie', 'uri:': 'http://www.wikidata.org/entity/Q219141'}, {'name': 'Cosmopolitan', 'uri:': 'http://www.wikidata.org/entity/Q268057'}]
